In [72]:
!pip install llama-index llama-index-llms-groq llama-index-vector-stores-faiss faiss-cpu llama-index-embeddings-huggingface -q

In [73]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [74]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir="/content/")
documents = reader.load_data()

In [75]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import StorageContext,VectorStoreIndex,KnowledgeGraphIndex,ServiceContext
import faiss

embeddings = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

vector_store = VectorStoreIndex.from_documents(documents=documents,embed_model=embeddings)

In [76]:
from llama_index.llms.groq import Groq

llm = Groq(model='llama-3.2-1b-preview')

In [77]:
from llama_index.core import PromptTemplate

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query}\n"
)

qa_template = PromptTemplate(template)

In [78]:
query_engine = vector_store.as_query_engine(llm=llm,prompt=qa_template)

In [79]:
query_engine.query('what is this document about?').response

'This document appears to be a resume for Mudassir Junejo, a Machine Learning Engineer with expertise in various areas, including computer vision, natural language processing, and predictive analytics.'